In [11]:
%matplotlib inline
import xarray as xr
from pyproj import Proj
import pandas as pd
import datetime as dt


#Streamflow gage station location in EPSG:8687
x,y = (160787, 6721261)

#Import precipitation and temperature dataset
path = "/home/Kristen/HydroElectric/Data/Climate/seNorge2018_2017.nc"
ds = xr.open_dataset(path)  # NetCDF or OPeNDAP URL

# Extract a dataset closest to specified point
dsloc = ds.sel(Y=y, X=x, method = 'nearest')
PT1=dsloc.to_dataframe()
PT1=PT1.reset_index(0).reset_index(drop=True)
PT1=PT1.drop("time_bnds", axis=1)
PT1 = PT1.drop_duplicates()
PT1["date"] = pd.to_datetime(PT1["time"]).dt.date


#Import shortwave radiation dataset
path = "/home/Kristen/HydroElectric/Data/Climate/HySN_Surface_Downwelling_Shortwave_Radiation_2017.nc"
ds = xr.open_dataset(path)  # NetCDF or OPeNDAP URL

# Extract a dataset closest to specified point
dsloc = ds.sel(Y=y, X=x, method = 'nearest')
PT2=dsloc.to_dataframe()
PT2["date"]= PT2.index
PT2["date"] = pd.to_datetime(PT2["date"]).dt.date
PT2=PT2.reset_index(drop=True)


#Import humidity dataset
path = "/home/Kristen/HydroElectric/Data/Climate/HySN_Near_Surface_Specific_Humidity_2017.nc"
ds = xr.open_dataset(path)  # NetCDF or OPeNDAP URL

# Extract a dataset closest to specified point
dsloc = ds.sel(Y=y, X=x, method = 'nearest')
PT3=dsloc.to_dataframe()
PT3["date"]= PT3.index
PT3["date"] = pd.to_datetime(PT3["date"]).dt.date
PT3=PT3.reset_index(drop=True)


/home/Kristen/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/Kristen/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/Kristen/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/Kristen/anaconda3/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_i

In [14]:
#Merge datasets together
df1 = pd.merge(PT1, PT2, on="date")
df2 = pd.merge(PT3, PT4, on="date")
df = pd.merge(df1, df2, on="date")
df=df[["date", "rr", "tn", "tx", "huss", "rsds"]]
df = df.rename(columns={'rr': 'precip', 'tn': 'temp_min', 'tx': 'temp_max', 'huss': 'humidity_spec', 'rsds': 'radiation_sw'})
df.to_csv("parameters.csv")
#Couldn't get date types to match to merge with streamflow data file so manually copid to streamflow datafile 
#for Jan 1, 2017 - Dec 31, 2017